# tests tab-NTV

In [69]:
import pandas as pd
log = {'file': 'IRVE_itinerance_residuel2023-03-03.csv',
      'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/Analyse/'}
irve = pd.read_csv(log['chemin'] + log['file'], sep=',', low_memory=False)
check = list(irve.columns)[17:]
regles = ['pdc non unique', 'station multi-opérateurs', 'station multi-enseignes', 'coordonnées non unique', 'pdc multi-stations'
         'station avec plusieurs noms', 'station multi-implantation', 'nombre de pdc par station incohérent', 
          'station multi-accès', 'station multi-horaires', 'accès deux-roues incohérent', 'coordonnées multi-adresses']
principal = [17, 18, 19, 20, 21, 22, 28]
secondaire = [23, 24, 25, 26, 27]
irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']

total = len(irve)
print(check, len(check))
for name in check:
    print(total - irve[name].sum())
print(total - irve['principal'].sum())
print(total - irve['secondaire'].sum())
print(total - irve['verif'].sum())

['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance', 'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance', 'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance', 'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance', 'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY'] 12
0
0
267
216
0
40
23
452
0
53
325
414
849
767
1616


In [70]:
import json

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
print(len(irve_p), len(irve_s))
#print(irve_p.columns)
popup = [[], []]
locat = [[], []]
for ind, irve in enumerate((irve_p, irve_s)):
    for i in range(len(irve)):
        popup[ind].append( {'id_station': irve['id_station_itinerance'][i], 
                           'id_pdc': irve['id_pdc_itinerance'][i],
                           'contact_operateur': irve['contact_operateur'][i],
                           'nom_enseigne': irve['nom_enseigne'][i],
                           'nom_station': irve['nom_station'][i],
                           'adresse_station': irve['adresse_station'][i],
                           'fichier des écarts': '<a href="https://github.com/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/">IRVE_itinerance_residuel</a>' } )
        #print(dic)
        coord = json.loads(irve['coordonneesXY'][i])
        coord.reverse()
        locat[ind].append(coord)
        #print(coord, type(coord))

363 105


In [71]:
from json_ntv import NtvSingle, NtvList, NtvSet, Ntv, NtvConnector
from json_ntv import NtvType, Namespace
from datetime import datetime, date, time
from util_carto import Cart

cart = Cart([50, 2.5])
cart.add_markers(locat[0], popup=popup[0], color='red', group='principal', max_width=250)
cart.add_markers(locat[1], popup=popup[1], color='orange', group='secondaire', max_width=250, icon='bug')
cart.show('irve.html')


In [ ]:
tab_data   = {'value':           [10, 20, 30, 40],
              'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21', '2022-01-22'],
              'value32::int32':  [10, 20, 30, 40],
              'coord::point':    [[1,2], [3,4], [5,6], [7,8]],
              'names::string':   ['john', 'eric', 'judith', 'max'],
              'index':           [1, 2, 3, 4],
              'complete_test':   [['a', 'b'], [0, 0, 1, 0]],
              'complete_date':   [{"::date": ["2000-01-01", "2000-02-01"]}, [0, 0, 1, 0]],
              'implicit_test':   [['a', 'b'], [1]],
              'implicit_test2':  [['a', 'b'], [2]],
              'unic_test':       'valunic' }
il = from_ntv_ilist(tab_data)
'''for idx in il.lindex:
    print('idx : ', idx.name, idx.codec, idx.keys)
for info in il.indexinfos():
    print('indexinfos : ', info)
print('\nil : ', il.lname, il.idxname, il.lvarname, il.primary) #, il.primaryname)
ntv = Ntv.obj(tab_data)
#print(ntv)
lidx = [list(decode_ntv_field(ntvf)) for ntvf in ntv]
print('\ntest', lidx, '\n\n')'''
for mode in ['full', 'default', 'optimize']:
    #print(to_ntv_ilist(il, mode))
    il2 = from_ntv_ilist(to_ntv_ilist(il, mode))
    print(mode, il == il2)
    #il2.analysis.actualize()
    if il != il2:
        print(to_ntv_ilist(il, mode))
        print(il.lvarname)
        print(il2.lvarname)